### Session 2: Analyzing Code Performance

#### Topics.
 1. Timing Julia functions
 2. Accurate benchmarking
 3. Profiling Julia functions
 4. (optional) Tracking detailed memory allocation

In [13]:
using Pkg
Pkg.activate(".")

  Activating new environment at `~/Documents/Github2/Physics-215-Julia/Session 2 - Measuring code performance/Project.toml`


**Session 2: Measuring code performance**

- Objective: Submit a performance analysis of a self-implemented physics textbook function or constant using Julia benchmarking tools.
- KR1: Implemented (customized) at least one math/physics textbook function, or constant (prefer those that involve a sum or a loop) in Julia. Discuss its importance in Physics. See Resources below.
- KR2: Compare the performance (accuracy) of the implemented function in comparison with the existing special functions within Julia.
- KR3: Successful loading of the `BenchmarkTools` module. May need to add it first via the `Pkg` or REPL package mode.
- KR4: Itemized differences between `@time`, `@btime`, `@benchmark` and other `@time`-like macros. Nice if the situations when they are best applied are mentioned.
- KR5:  Identified demonstrated useful features within the `Profiler` module of Julia. Features must be explained why useful for your case.
- KR6: A discussion of the performance of the implemented function above.
- KR7: Disuss the computational complexity of the Madelbrot function you made based on KR5. What is the best `@time` output to use for this?

In [2]:
"""
    seql_sum(x)

A naive cumulative function.
Input: iterable container `x`
Output: sum of all elements of `x`
"""
function seql_sum(x)
    result = zero(eltype(x))
    for i in eachindex(x)
        result += x[i]
    end
    return result
end

seql_sum

In [4]:
N = 10_000_000
x64 = rand(N) #Float64 type
x32 = rand(Float32, 2*length(x64)); #same memory length

In [6]:
println("\nNOTES:")
println("> eps(eltype(x64)) is $(round(eps(eltype(x64)), digits = 20)).");
total = 1/sqrt(length(x64))
println("> Statistical tolerance is  $(round(total, digits = 20)).");


NOTES:
> eps(eltype(x64)) is 2.2204e-16.
> Statistical tolerance is  0.00031622776601683794.


In [7]:
#Timing seql_sum()

base64 = @timed sum(x64);
base64 = @timed sum(x64);

base32 = @timed sum(x32);
base32 = @timed sum(x32);

typeof(base64)

NamedTuple{(:value, :time, :bytes, :gctime, :gcstats), Tuple{Float64, Float64, Int64, Float64, Base.GC_Diff}}

In [12]:
t64 = base64[:time]
t32 = base32[:time]

println("The time t64 for sum(x64) is $(t64).")
println("The time t32 for sum(x32) is $(t32).")
println("     such that t32/t64 = $(round((t32/t64), digits = 5)).")
println("               t64/t64 = $(round((t64/t64), digits = 5)).")
println("     and       t32/t32 = $(round((t32/t32), digits = 5)).")

The time t64 for sum(x64) is 0.005808245.
The time t32 for sum(x32) is 0.005970829.
     such that t32/t64 = 1.02799.
               t64/t64 = 1.0.
     and       t32/t32 = 1.0.


In [14]:
using BenchmarkTools

In [15]:
?@btime

```
@btime expression [other parameters...]
```

Similar to the `@time` macro included with Julia, this executes an expression, printing the time it took to execute and the memory allocated before returning the value of the expression.

Unlike `@time`, it uses the `@benchmark` macro, and accepts all of the same additional parameters as `@benchmark`.  The printed time is the *minimum* elapsed time measured during the benchmark.


In [16]:
@btime seql_sum($x64)

  11.790 ms (0 allocations: 0 bytes)


5.000127522141715e6

In [17]:
?@benchmark

```
@benchmark <expr to benchmark> [setup=<setup expr>]
```

Run benchmark on a given expression.

# Example

The simplest usage of this macro is to put it in front of what you want to benchmark.

```julia-repl
julia> @benchmark sin(1)
BenchmarkTools.Trial:
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     13.610 ns (0.00% GC)
  median time:      13.622 ns (0.00% GC)
  mean time:        13.638 ns (0.00% GC)
  maximum time:     21.084 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     998
```

You can interpolate values into `@benchmark` expressions:

```julia
# rand(1000) is executed for each evaluation
julia> @benchmark sum(rand(1000))
BenchmarkTools.Trial:
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     1.566 μs (0.00% GC)
  median time:      2.135 μs (0.00% GC)
  mean time:        3.071 μs (25.06% GC)
  maximum time:     296.818 μs (95.91% GC)
  --------------
  samples:          10000
  evals/sample:     10

# rand(1000) is evaluated at definition time, and the resulting
# value is interpolated into the benchmark expression
julia> @benchmark sum($(rand(1000)))
BenchmarkTools.Trial:
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     101.627 ns (0.00% GC)
  median time:      101.909 ns (0.00% GC)
  mean time:        103.834 ns (0.00% GC)
  maximum time:     276.033 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     935
```


In [18]:
bench = @benchmark seql_sum($x64)

BenchmarkTools.Trial: 363 samples with 1 evaluation.
 Range (min … max):  11.769 ms … 19.480 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.493 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.770 ms ±  1.376 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▄▇▆▄▅▁▂▁█▄▄▆▂▅     ▁                                    
  ▅▄▆█▇███████████████▆██▅█▄▆▆▄▃▅▆▃▅▃▃▃▃▃▁▃▃▃▁▁▁▄▁▁▃▁▁▁▃▁▃▃▁▃ ▄
  11.8 ms         Histogram: frequency by time          19 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [19]:
?propertynames

search: propertynames



```
propertynames(x, private=false)
```

Get a tuple or a vector of the properties (`x.property`) of an object `x`. This is typically the same as [`fieldnames(typeof(x))`](@ref), but types that overload [`getproperty`](@ref) should generally overload `propertynames` as well to get the properties of an instance of the type.

`propertynames(x)` may return only "public" property names that are part of the documented interface of `x`.   If you want it to also return "private" fieldnames intended for internal use, pass `true` for the optional second argument. REPL tab completion on `x.` shows only the `private=false` properties.


In [20]:
propertynames(bench)

(:params, :times, :gctimes, :memory, :allocs)